### Biliotecas

In [ ]:
! pip install pandas
! pip install numpy
! pip install scikit-learn

In [ ]:
#Bibliotecas para Manipulação e Acesso dos Dados
import pandas as pd
import numpy as np

#Bibliotecas para Métricas
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

#Biblioteca para o Modelo
from statsmodels.tsa.statespace.sarimax import SARIMAX

#Bibliotecas para Interface
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

## Modelo SARIMAX Geral

O modelo SARIMAX é um modelo para previsões em séries temporais, combinando autoregressão, média móvel e diferenciação sazonal. Ele se destaca por lidar bem com dados que possuem tendências, sazonalidades e fatores externos, tornando-o ideal para prever receitas financeiras. Além de capturar padrões temporais complexos, o SARIMAX também pode incorporar variáveis externas que afetam o comportamento da receita, permitindo previsões mais precisas e ajustadas ao contexto do negócio.

### Carregamento de Dados

In [ ]:
df = pd.read_excel('/content/tabela_data_sarimax.xlsx')

### Limpeza de Dados

In [ ]:
#Converte a coluna que têm vírgulas para o formato correto
df.loc[:,'Taxa_Ac._TRI_%_PIB'] = df['Taxa_Ac._TRI_%_PIB'].str.replace(',', '.').astype(float)

#Preenche as células vásias com a média
df['Vl_Liquido_Final'] = df['Vl_Liquido_Final'].fillna(df['Vl_Liquido_Final'].mean())
df['IPCA_ES'] = df['IPCA_ES'].fillna(df['IPCA_ES'].mean())
df['IPCA_BR'] = df['IPCA_BR'].fillna(df['IPCA_BR'].mean())
df['Taxa_Ac._TRI_%_PIB'] = df['Taxa_Ac._TRI_%_PIB'].fillna(df['Taxa_Ac._TRI_%_PIB'].mean())

#Converta as colunas para o tipo numérico
df['Vl_Liquido_Final'] = pd.to_numeric(df['Vl_Liquido_Final'], errors='coerce')
df['IPCA_ES'] = pd.to_numeric(df['IPCA_ES'], errors='coerce')
df['IPCA_BR'] = pd.to_numeric(df['IPCA_BR'], errors='coerce')
df['Taxa_Ac._TRI_%_PIB'] = pd.to_numeric(df['Taxa_Ac._TRI_%_PIB'], errors='coerce')

#Converte as colunas de data para o formato de data
df['data'] = pd.to_datetime(df['data'])
df.set_index('data', inplace=True)

### Seleção de Váriaveis

In [ ]:
#Variáveis de treinamento e Variável Alvo 
X = df[['IPCA_ES', 'IPCA_BR', 'Taxa_Ac._TRI_%_PIB']]
y = df['Vl_Liquido_Final']

#Separar os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

### Modelagem com SARIMAX

In [ ]:
#Ajustar o modelo
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 12)  #Sazonalidade mensal
model = SARIMAX(y_train, exog=X_train, order=order, seasonal_order=seasonal_order)
model_fit = model.fit(disp=False)

#Fazer previsões
y_pred = model_fit.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, exog=X_test)

#Métricas do modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
smape = np.mean(2 * np.abs(y_pred - y_test) / (np.abs(y_pred) + np.abs(y_test))) * 100

#Exibir as métricas
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R²: {r2}')
print(f'SMAPE: {smape}')

#Previsões para o próximo ano
n_months = 12
future_dates = pd.date_range(start=df.index[-1] + pd.DateOffset(months=1), periods=n_months, freq='ME')
future_X = pd.DataFrame(index=future_dates)

# Preencher as variáveis de treinamento futuras com base na média dos dados passados
for col in X.columns:
    future_X[col] = X[col].mean()

#Fazer mais previsões futuras
future_predictions = model_fit.predict(start=len(y_train), end=len(y_train) + n_months - 1, exog=future_X)

print("Previsões futuras para o próximo ano:")
print(future_predictions)

## Interface

### Simulação de Modelo

In [ ]:
#Simulando um modelo preditivo
def modelo_preditivo(veiculo):
    #Simulação para os próximos 12 meses
    np.random.seed(42)
    receitas = np.random.uniform(50000, 150000, size=12)
    return receitas

### Gerar a predição a partir do Modelo

In [ ]:
#Gerar a previsão e mostrar os dados futuros
def fazer_previsao(veiculo):
    receitas = modelo_preditivo(veiculo)
    meses = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    #Plotar o gráfico de receitas
    plt.figure(figsize=(10, 6))
    plt.plot(meses, receitas, marker='o')
    plt.title(f'Previsão de Receita para o Veículo: {veiculo}')
    plt.xlabel('Meses')
    plt.ylabel('Receita (R$)')
    plt.grid(True)
    plt.show()

    #Mostrar as previsões em texto
    print(f"Previsão de Receita Detalhada para o Veículo: {veiculo}")
    for i, receita in enumerate(receitas):
        print(f"Mês {i+1} ({meses[i]}): R$ {receita:,.2f}")

### Interface

#### Seletor de UEN

In [ ]:
veiculos = ['TV', 'Rádio', 'Jornal', 'Online']
seletor_veiculo = widgets.Dropdown(
    options=veiculos,
    value='TV',
    description='Veículo:',
)

#### Botão

In [ ]:
botao_previsao = widgets.Button(description="Fazer Previsão")

def ao_clicar_botao(b):
    veiculo_escolhido = seletor_veiculo.value
    fazer_previsao(veiculo_escolhido)

botao_previsao.on_click(ao_clicar_botao)

#### Display

In [ ]:
display(seletor_veiculo, botao_previsao)